# Exploratory Data Analysis 

## Performed on the energy network dataset, to verify outliers, distribution and meaningful graphs

In [2]:
# pip install sweetviz
# conda install -c conda-forge lux-api

# Specific libraries
# import sweetviz as sv
# import autoviz
# from autoviz.AutoViz_Class import AutoViz_Class
import lux

# General libraries
import sys, os
from os import system
import warnings

import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt

from datetime import datetime as dt
from datetime import date
from datetime import timedelta

### Part 0: Integrating the data

In [3]:
# Constants
values_column_names = ["time", "branch" , "organization", "substation", "transformer_code", "App SW", 
                        "V_L1", "I_L1", "W_L1", "QL_L1", "QC_L1","cos_L1", "angle_L1",
                        "V_L2", "I_L2", "W_L2", "QL_L2", "QC_L2","cos_L2", "angle_L2",
                        "V_L3", "I_L3", "W_L3", "QL_L3", "QC_L3","cos_L3", "angle_L3",
                        "temp_amb",
                        "aplus_L1", "aminus_L1", "RplusL_L1", "RminusL_L1", "RplusC_L1", "RminusC_L1", 
                        "aplus_L2", "aminus_L2", "RplusL_L2", "RminusL_L2", "RplusC_L2", "RminusC_L2",
                        "aplus_L3", "aminus_L3", "RplusL_L3", "RminusL_L3", "RplusC_L3", "RminusC_L3"]

script_path = os.getcwd()
data = pd.read_excel(script_path + '/../DATA/LVSM_Def.xlsx',  engine='openpyxl', header=None, names=values_column_names)

# Cleaning data table
data = data.drop([0], axis = 0)        # Dropping the headers
data = data.drop(["aminus_L1", "RminusL_L1", "RplusC_L1", 
                  "aminus_L2", "RminusL_L2", "RplusC_L2",
                  "aminus_L3", "RminusL_L3", "RplusC_L3"], axis=1)

# data = data.drop(["aplus_L1", "aminus_L1", "RplusL_L1", "RminusL_L1", "RplusC_L1", "RminusC_L1", 
#                  "aplus_L2", "aminus_L2", "RplusL_L2", "RminusL_L2", "RplusC_L2", "RminusC_L2",
#                  "aplus_L3", "aminus_L3", "RplusL_L3", "RminusL_L3", "RplusC_L3", "RminusC_L3"], axis=1)

data = data.reset_index(drop = True)


In [4]:
# Copy of the dataframe to split date and hour
data_new = data.copy(deep=True)

In [5]:
### Preformat

### Deal with the "24:00" problem. Adapt BOTH the hour and the day.
# Get the indexes and replace hour
for i, date in enumerate(data_new['time']):
    if date.split()[1].split(':')[0] == '24':
        data_new.loc[i]['time'] = data_new.loc[i]['time'].replace("24:00","00:00")
        data_new.loc[i]['time'] = pd.to_datetime(data_new.loc[i]['time'], format = '%Y-%m-%d %H:%M') + timedelta(days = 1)

# Update the format
data_new['time'] = pd.to_datetime(data_new['time'], format = '%Y-%m-%d %H:%M:%S')

# Split the time column into date and hour columns, for diagram's input preparation
data_new['date'] = (data_new['time']).dt.date
data_new['hour'] = (data_new['time']).dt.time

# Delete the old time column
data_new = data_new.drop(["time"], axis=1)

# Put both columns at the start
data_new = pd.concat([data_new['hour'], data_new.drop('hour',axis=1)], axis=1)
data_new = pd.concat([data_new['date'], data_new.drop('date',axis=1)], axis=1)

# Cleaning and slicing
if data_new.isna().sum().sum() < .10 * len(data_new): 
    print ("Cleaning NA values from dataset")
    data_new = data_new.dropna()
else:
    raise Exception("Careful! Deleting NaN values would cut most of the dataset")



Cleaning NA values from dataset


In [8]:
warnings.filterwarnings('ignore')
data_new.head()

,date,hour,branch,organization,substation,transformer_code,App SW,V_L1,I_L1,W_L1,...,temp_amb,aplus_L1,RplusL_L1,RminusC_L1,aplus_L2,RplusL_L2,RminusC_L2,aplus_L3,RplusL_L3,RminusC_L3
0,2019-06-16,01:00:00,AE,SZZ,S201,TR1,003F,234,65,14964,...,30,16082,1983,0,16736,1620,0,23015,2179,0
1,2019-06-16,02:00:00,AE,SZZ,S201,TR1,003F,233,57,13091,...,29,14342,1441,0,14545,1057,28,23764,2906,0
2,2019-06-16,03:00:00,AE,SZZ,S201,TR1,003F,236,55,12847,...,29,13543,1381,0,14073,1141,0,22147,2942,0
3,2019-06-16,04:00:00,AE,SZZ,S201,TR1,003F,234,135,30517,...,29,20757,2954,0,22059,2021,0,27317,3701,0
4,2019-06-16,05:00:00,AE,SZZ,S201,TR1,003F,235,102,23069,...,29,29753,5054,0,31259,3121,2,33013,3778,0


In [9]:
data.describe()

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [74]:
# Prepare the train and test dataset
msk = np.random.rand(len(data_new)) < 0.98

df_train = data_new[msk]
df_test = data_new[~msk]

### Part 1: Automating EDA - Using Sweetviz

In [75]:
# Create analysis report
analyze_report = sv.analyze(data_new)
analyze_report.show_html()

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)
Report SWEETVIZ_REPORT.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
# Create a Train and Test Data Comparison - Differences and Similarities

compare = sv.compare([df_train, "Training Data"], [df_test, "Test Data"], "W_L1")
compare.show_html()

In [77]:
# Create a Train and Test Data Comparison - Particular column

intra_com = sv.compare_intra(df_train, df_train["App SW"] == "083E", ["083E", "003F", "0A3E", "093E", "00BF", "0B3E", "081E", "082E", "09BE", "0818"])
intra_com.show_html(filepath='Compare_Intra.html', open_browser=True, layout='widescreen', scale=None)

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:01 -> (00:00 left)
Report Compare_Intra.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
# Display the info on the output cell, not the browser
# report.show_notebook(w=None, h=None, scale=None,layout='widescreen',filepath=None)

### Part 2: Automating EDA - Using Autoviz

In [83]:
AV = AutoViz_Class()

df1 = AV.AutoViz(script_path + '/../DATA/LVSM_Def.xlsx')

Shape of your Data Set: (167548, 46)
############## C L A S S I F Y I N G  V A R I A B L E S  ####################
Classifying variables in data set...
    Number of Numeric Columns =  31
    Number of Integer-Categorical Columns =  0
    Number of String-Categorical Columns =  2
    Number of Factor-Categorical Columns =  0
    Number of String-Boolean Columns =  0
    Number of Numeric-Boolean Columns =  5
    Number of Discrete String Columns =  1
    Number of NLP String Columns =  0
    Number of Date Time Columns =  0
    Number of ID Columns =  0
    Number of Columns to Delete =  7
    46 Predictors classified...
        This does not include the Target column(s)
        8 variables removed since they were ID or low-information variables
Since Number of Rows in data 167548 exceeds maximum, randomly sampling 150000 rows for EDA...
30 numeric variables in data exceeds limit, taking top 30 variables
Number of All Scatter Plots = 465


KeyboardInterrupt: 

### Part 3: Automating EDA - Using Lux

In [11]:
data_new

,date,hour,branch,organization,substation,transformer_code,App SW,V_L1,I_L1,W_L1,...,temp_amb,aplus_L1,RplusL_L1,RminusC_L1,aplus_L2,RplusL_L2,RminusC_L2,aplus_L3,RplusL_L3,RminusC_L3
0,2019-06-16,01:00:00,AE,SZZ,S201,TR1,003F,234,65,14964,...,30,16082,1983,0,16736,1620,0,23015,2179,0
1,2019-06-16,02:00:00,AE,SZZ,S201,TR1,003F,233,57,13091,...,29,14342,1441,0,14545,1057,28,23764,2906,0
2,2019-06-16,03:00:00,AE,SZZ,S201,TR1,003F,236,55,12847,...,29,13543,1381,0,14073,1141,0,22147,2942,0
3,2019-06-16,04:00:00,AE,SZZ,S201,TR1,003F,234,135,30517,...,29,20757,2954,0,22059,2021,0,27317,3701,0
4,2019-06-16,05:00:00,AE,SZZ,S201,TR1,003F,235,102,23069,...,29,29753,5054,0,31259,3121,2,33013,3778,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167543,2020-06-08,10:00:00,AE,SZZ,S868,TR1,083E,235,113,25752,...,23,27927,7906,35,28051,4700,125,24854,4588,162
167544,2020-06-08,11:00:00,AE,SZZ,S868,TR1,093E,235,64,15021,...,23,15483,394,737,13628,24,4236,10960,0,5205
167545,2020-06-08,12:00:00,AE,SZZ,S868,TR1,083E,234,152,-31174,...,23,21869,2860,489,18482,1391,3352,15732,1391,3942
167546,2020-06-08,13:00:00,AE,SZZ,S868,TR1,083E,234,164,-28258,...,23,35488,9220,0,33392,5804,0,29780,5657,0


In [ ]:
data_new.intent = ["I_L1", "W_L1"]
data_new

In [ ]:
interestingVis = data_new.exported
interestingVis

In [ ]:
interestingVis[0]

In [ ]:
X.recommendation["Enhance"]

In [ ]:
print(interestingVis[0].to_matplotlib())

### Part 4: Filter useful info

In [ ]:
data = data.drop(["aminus_L1", "RminusL_L1", "RplusC_L1", 
                  "aminus_L2", "RminusL_L2", "RplusC_L2",
                  "aminus_L3", "RminusL_L3", "RplusC_L3"], axis=1)


In [ ]:
# Randomly shuffle a dataframe
data.reindex(np.random.permutation(data.index))